In [84]:
import pandas as pd
import os
import jieba
from tqdm import tqdm
from rouge_chinese import Rouge
from dotenv import load_dotenv
from zhipuai import ZhipuAI
from chromadb import PersistentClient
load_dotenv()
client = ZhipuAI(api_key=os.getenv("ZHIPUAI_API_KEY"))
client_chroma = PersistentClient("chroma")
collection = client_chroma.get_collection(name="plum")

In [76]:
df_chunks = pd.read_csv("preprocessed/chunks.csv", sep="|", index_col=0)
df_chunks

,text,source,page
0,李产业发展现状\n一、中国李产业现状\n(一)栽培历史\n我国李栽培历史久远，原始居民就有采...,./李高效生产及绿色防控技术.pdf,0
1,李高效生产及绿色防控技术\n考证，“藉栽”指自根繁殖，即包括扦插、压条、分株等无性繁殖方\n...,./李高效生产及绿色防控技术.pdf,1
2,"第一章李产业发展现状\n年积温在4500~8000℃,最冷月平均气温0~15℃,是热带与温带...",./李高效生产及绿色防控技术.pdf,2
3,李高效生产及绿色防控技术\n(3)资源品种。本区主要栽培种为中国李，部分为欧洲李、杏李\n和...,./李高效生产及绿色防控技术.pdf,3
4,第一章李产业发展现状\n2绥棱红(北方1号)，它的母本和父本分别来自于小黄李和福摩萨，\n由...,./李高效生产及绿色防控技术.pdf,4
...,...,...,...
145,第五章李贮运保鲜与加工\n(5)预煮。预煮前用1.5%的食盐水护色，最后用清水淘洗1次。\n...,./李高效生产及绿色防控技术.pdf,145
146,李高效生产及绿色防控技术\n2.主要加工设施设备\n(1)清洗设备。鼓泡清洗机、毛刷清洗机、...,./李高效生产及绿色防控技术.pdf,146
147,第五章李贮运保鲜与加工\n4.产品质量要求\n(1)感官指标。具有该品种李加工后应有的正常颜...,./李高效生产及绿色防控技术.pdf,147
148,李高效生产及绿色防控技术\n(3)清洗。采用果蔬专用清洗设备清洗表面灰尘、腐叶、虫卵等\n污...,./李高效生产及绿色防控技术.pdf,148


In [77]:
df_qa = pd.read_csv("preprocessed/qa.csv", sep="|", index_col=0)
df_qa

,q,a,d
0,请问中国李产业的历史可以追溯到什么时候？,远在5000多年前,0
1,中国最早关于李的文献记载出现在哪部古籍中？,《诗经》,0
2,《广志》记载汉代上林苑收集了多少个李品种？,15个,0
3,《尔雅》中记录了哪些李品种？,椄虑李、无实李、赤李,0
4,《山海经》中提到的灵山有哪些果树？,桃、李、梅、杏,0
...,...,...,...
838,李高效生产及绿色防控技术的具体步骤是怎样的？,李高效生产及绿色防控技术的步骤包括清洗、破碎打浆、加热浓缩、洗瓶、装罐及密封和杀菌冷却。,148
839,在李果酱的生产过程中，原浆是如何处理的？,在李果酱的生产过程中，原浆是通过破碎打浆机将李打成原浆，并在打浆过程中添加0.2%~0.5%...,148
840,李果酱在装罐封口时的温度要求是多少？,李果酱在装罐封口时的温度要求在80℃以上，以防止微生物污染。,148
841,李果酱的产品质量要求有哪些？,李果酱的产品质量要求包括感官指标、理化指标、污染物限量和微生物限量。感官指标要求具有该产品应...,148


In [95]:
SYSTEM_PROMPT_EVAL_RAG = """根据给定上下文，正确地问答问题，只需要输出答案，不需要输出解答过程。"""
SYSTEM_PROMPT_EVAL_VANILLA = """正确地问答问题，只需要输出答案，不需要输出解答过程。"""
SYSTEM_PROMPT_EVAL_CGRAG = """"""

METHOD = "cheat"

SYSTEM_PROMPTS = {
  "vanilla": SYSTEM_PROMPT_EVAL_VANILLA,
  "rag": SYSTEM_PROMPT_EVAL_RAG,
  "cg-rag": SYSTEM_PROMPT_EVAL_CGRAG,
  "cheat": SYSTEM_PROMPT_EVAL_RAG
}


def llm(text: str):
  res = client.chat.completions.create(
    model="glm-3-turbo",
    messages=[
      {"role": "system", "content": SYSTEM_PROMPTS[METHOD]},
      {"role": "user", "content": text},
    ]
  )
  return res.choices[0].message

In [87]:
with open(f"eval/glm-3-turbo-{METHOD}.csv", "a") as w:
  for i, row in tqdm(df_qa.iterrows()):
    if i >= 0:
      q = row["q"]
      a = row["a"]
      d = row["d"]
      match METHOD:
        case "vanilla":
          content = llm(f"问题: {q}").content
        case "rag":
          chunks = collection.query(
            query_texts=[q],
            n_results=3
          )["documents"]
          context = "\n\n".join([c[0] for c in chunks])
          content = llm(f"问题: {q}\n上下文: {context}").content
        case "cheat":
          context = df_chunks.iloc[i]["text"]
          content = llm(f"问题: {q}\n上下文: {context}").content

      w.write(f"{q}|{a}|{content}\n")
      w.flush()

843it [28:40,  2.04s/it]


In [96]:
df_glm_3_turbo = pd.read_csv(f"eval/glm-3-turbo-{METHOD}.csv", sep="|", names=["q", "hyp", "ref"]).dropna().reset_index(drop=True)
df_glm_3_turbo

,q,hyp,ref
0,请问中国李产业的历史可以追溯到什么时候？,远在5000多年前,中国李产业的历史可以追溯到先秦时期。
1,中国最早关于李的文献记载出现在哪部古籍中？,《诗经》,《尔雅》。
2,《广志》记载汉代上林苑收集了多少个李品种？,15个,答案: 15个。
3,《尔雅》中记录了哪些李品种？,椄虑李、无实李、赤李,《尔雅》中记载有“椄虑李”“无实李”和“赤李”3个品种。
4,《山海经》中提到的灵山有哪些果树？,桃、李、梅、杏,灵山中的果树有桃、李、梅、杏。
...,...,...,...
838,李高效生产及绿色防控技术的具体步骤是怎样的？,李高效生产及绿色防控技术的步骤包括清洗、破碎打浆、加热浓缩、洗瓶、装罐及密封和杀菌冷却。,具体步骤包括：
839,在李果酱的生产过程中，原浆是如何处理的？,在李果酱的生产过程中，原浆是通过破碎打浆机将李打成原浆，并在打浆过程中添加0.2%~0.5%...,原浆是通过破碎打浆机将李打成，同时添加0.2%~0.5%的D-异抗坏血酸钠、0.04‰~0....
840,李果酱在装罐封口时的温度要求是多少？,李果酱在装罐封口时的温度要求在80℃以上，以防止微生物污染。,封口时果酱温度在80℃以上。
841,李果酱的产品质量要求有哪些？,李果酱的产品质量要求包括感官指标、理化指标、污染物限量和微生物限量。感官指标要求具有该产品应...,产品质量要求包括：


In [99]:
SYSTEM_PROMPT_JUDGE = """你是一个问答题评判专家，你的任务是评判模型输出是否和参考答案一致。

如果完全一致，则输出 1
如果部分匹配，则输出 0.5
如果意思冲突，则输出 0

注意用词不一样但意思完全一样的情况也输出 1。
只需要输出结果，不需要输出分析过程。
"""
def judge(text: str):
  res = client.chat.completions.create(
    model="glm-3-turbo",
    messages=[
      {"role": "system", "content": SYSTEM_PROMPT_JUDGE},
      {"role": "user", "content": text},
    ]
  )
  return res.choices[0].message

In [100]:
with open(f"eval/glm-3-turbo-{METHOD}-eval.csv", "a") as w:
    for i, row in tqdm(df_glm_3_turbo.iterrows()):
        if i >= 0:
            hyp = row["hyp"]
            ref = row["ref"]
            content = judge(f"模型输出: {hyp}\n参考答案{ref}").content
            w.write(f"{content}\n")
            w.flush()

843it [09:39,  1.45it/s]


In [101]:
df_glm_3_turbo_eval = pd.read_csv(f"eval/glm-3-turbo-{METHOD}-eval.csv", names=["score"]).dropna()
df_glm_3_turbo_eval["score"].sum() / len(df_glm_3_turbo_eval)

0.6432977461447211

In [94]:
df_glm_3_turbo["acc"] = df_glm_3_turbo_eval["score"]
df_glm_3_turbo

,q,hyp,ref,acc
0,请问中国李产业的历史可以追溯到什么时候？,远在5000多年前,20世纪五六十年代。,0.0
1,中国最早关于李的文献记载出现在哪部古籍中？,《诗经》,根据给定的上下文，没有提到中国最早关于李的文献记载出现在哪部古籍中。上下文主要讲述的是中国李...,0.0
2,《广志》记载汉代上林苑收集了多少个李品种？,15个,上下文中没有提到《广志》记载汉代上林苑收集了多少个李品种的具体信息。,0.0
3,《尔雅》中记录了哪些李品种？,椄虑李、无实李、赤李,上下文中没有直接提到《尔雅》中记录的李品种。,0.0
4,《山海经》中提到的灵山有哪些果树？,桃、李、梅、杏,答案: 上下文中没有提到《山海经》中灵山的果树信息。,0.0
...,...,...,...,...
836,李高效生产及绿色防控技术的具体步骤是怎样的？,李高效生产及绿色防控技术的步骤包括清洗、破碎打浆、加热浓缩、洗瓶、装罐及密封和杀菌冷却。,根据给定的上下文，李高效生产及绿色防控技术的具体步骤没有直接描述。上下文主要提供了李果酒的生...,0.0
837,在李果酱的生产过程中，原浆是如何处理的？,在李果酱的生产过程中，原浆是通过破碎打浆机将李打成原浆，并在打浆过程中添加0.2%~0.5%...,原浆在李果酱的生产过程中的处理工艺没有在上下文中直接提及。上下文主要介绍了李果酒的生产过程，...,0.0
838,李果酱在装罐封口时的温度要求是多少？,李果酱在装罐封口时的温度要求在80℃以上，以防止微生物污染。,对不起，您提供的上下文信息中并没有提到李果酱在装罐封口时的温度要求。请提供更多的上下文信息，...,0.0
839,李果酱的产品质量要求有哪些？,李果酱的产品质量要求包括感官指标、理化指标、污染物限量和微生物限量。感官指标要求具有该产品应...,答案: 产品质量要求包括栽培的距离、嫁接繁殖的砧木选择、对环境条件的调整（如避免连阴、过燥或...,0.0


In [97]:
hyps: list[str] = []
refs: list[str] = []

for i, row in df_glm_3_turbo.iterrows():
    hyp = row["hyp"]
    ref = row["ref"]
    hyps.append(" ".join(jieba.cut(hyp)))
    refs.append(" ".join(jieba.cut(ref)))

print(len(hyps), len(refs))

843 843


In [98]:
rouge = Rouge()
rouge.get_scores(hyps, refs, avg=True)

{'rouge-1': {'r': 0.5454916761038325,
  'p': 0.7163136663807729,
  'f': 0.553581695932416},
 'rouge-2': {'r': 0.42424072149622527,
  'p': 0.5781502576531354,
  'f': 0.4330276653433266},
 'rouge-l': {'r': 0.5258487860813746,
  'p': 0.6958760900159118,
  'f': 0.5303710370827815}}

In [108]:
rouge.get_scores("我 谁 呢", "我 是 谁 呀")

[{'rouge-1': {'r': 0.5, 'p': 0.6666666666666666, 'f': 0.5714285665306124},
  'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0},
  'rouge-l': {'r': 0.5, 'p': 0.6666666666666666, 'f': 0.5714285665306124}}]

In [110]:
4/7

0.5714285714285714